# TESTS

In [ ]:
from pathlib import Path
import os
from modflowapi.extensions import ApiSimulation
from modflowapi import Callbacks
from workflow import *
from datetime import datetime

DT_FMT = "%Y-%m-%d %H:%M:%S"


In [ ]:
mf, mt = build_mt3dms_model(sim_name, ws = ws)
# run_mt3dms(mt)

In [ ]:
# from shutil import which
# print(which('/mf2005'))
# ?flopy.modflow.Modflow

# m

In [ ]:
sim_names = ['mt3dms', 'mf6gwt', 'mf6gwtapi']
ws = os.path.join('benchmark')

fig, axs = plt.subplots(3, 1, figsize=(6.3,10.2))

for sim_name, ax in zip(sim_names, axs.flatten()):
    print(sim_name)
    if sim_name == 'mt3dms':
        ucname = 'MT3D001'
    else:
        ucname = 'tracer'
    ucn = flopy.utils.HeadFile(os.path.join(ws, sim_name, f"{ucname}.UCN"),text=f"concentration")
    results = ucn.get_alldata()
    times = ucn.get_times()
    r_tf = results[-1, 0, 0,:]
    ax.plot(r_tf)

In [ ]:
sim_names = ['mt3dms', 'mf6gwt', 'mf6gwtapi']
ws = os.path.join('benchmark')

x = np.arange(0.005,0.505,0.01)

fig, axs = plt.subplots(1, 1, figsize=(6.3,3.3))
ax = axs
for e, sim_name in enumerate(sim_names):
    if sim_name == 'mt3dms':
        ucname = 'MT3D001'
    else:
        ucname = 'tracer'
    ucn = flopy.utils.HeadFile(os.path.join(ws, sim_name, f"{ucname}.UCN"),text=f"concentration")
    results = ucn.get_alldata()
    times = ucn.get_times()
    r_tf = results[-1, 0, 0,:]
    ax.plot(x, r_tf, label = sim_name, linewidth= 5-e*1.5, alpha = 0.6+e/5)
    ax.legend()
    ax.set_ylabel('Conc mol (m$^{-3}$)')
    ax.set_xlabel('Distance (m)')
fig.tight_layout()

In [ ]:
sim_name = 'mf6gwt'
ws = os.path.join('benchmark')
ucn = flopy.utils.HeadFile(os.path.join(ws,sim_name, f"tracer.UCN"),text=f"concentration")
# ucn = flopy.utils.UcnFile(os.path.join(ws,sim_name, f"MT3D001.UCN"))
results = ucn.get_alldata()
results
times = ucn.get_times()

# fig, axs = plt.subplots(1, 1, figsize=(6.3,3.2))
ax = axs[1]
r_tf = results[-1, 0, 0,:]
ax.plot(r_tf)


In [ ]:
sim_name = 'mf6gwtapi'
ws = os.path.join('benchmark')
ucn = flopy.utils.HeadFile(os.path.join(ws,sim_name, f"tracer.UCN"),text=f"concentration")
# ucn = flopy.utils.UcnFile(os.path.join(ws,sim_name, f"MT3D001.UCN"))
results = ucn.get_alldata()
results
times = ucn.get_times()

fig, axs = plt.subplots(1, 1, figsize=(6.3,3.2))
ax = axs
r_tf = results[-1, 0, 0,:]
ax.plot(r_tf)

In [ ]:
sim_name = 'engesgaard1992'
initsol_components, sconc_init = init_solution(init_file = 'initsol.dat')

q = 0.259
wel_rec = wel_array(q, sconc_init, aux = True)
components, phreeqc_rm, sconc = initialize_phreeqcrm(sim_name)    


sim = build_model(ws = 'model', sim_name = sim_name, spls = components, 
                    sconc=sconc, wel_rec=wel_rec, init_comp=sconc_init)


# run_model(sim)
# plot_heads(sim)
# plot_concentrations(sim)

sim_ws = Path("model/engesgaard1992/")
dll = Path("bin/win/libmf6")
resultsapi = mf6rtm_api_test(dll, sim_ws, components=components, phreeqc_rm=phreeqc_rm, reaction = False)

In [ ]:
time = np.arange(0,0.24, 0.01)

resultsapi = np.array(resultsapi)

resultsapi.shape

resultsapi[:,-1,-1]
fig, axs = plt.subplots(1, 1, figsize=(6.3, 3.2))
ax = axs

ax.plot(time, resultsapi[:,-1,-1])

In [ ]:
sim = build_model(ws = 'model', sim_name = sim_name, spls = components, 
                    sconc=sconc, wel_rec=wel_rec, init_comp=sconc_init)


run_model(sim)


In [ ]:
model_name = 'Mg'
ucn = flopy.utils.HeadFile(os.path.join("model",sim.name,f"{model_name}.ucn"),text="concentration")
results = ucn.get_alldata()
fig, axs = plt.subplots(1, 1, figsize=(6.3, 3.2))
ax = axs

ax.plot(time, results[:,0,0,-2])
# ax.set_ylim(0,200)

In [ ]:
results[:,0,0,0]

In [ ]:
# sim_ws = os.path.join('model', '1dtest', 'chloride')
sim_ws = Path("model/engesgaard1992/")
dll = Path("bin/win/libmf6")

In [ ]:
def try_get_value(address, mf6):
    try:
        res = mf6.get_value(address)
        return res
    except:
        res = 'failed'
        return res

In [ ]:
mf6 = modflowapi.ModflowApi(dll, working_directory = sim_ws)
mf6.initialize()
# current_time = mf6.get_current_time()
# end_time = mf6.get_end_time()
all_vars = mf6.get_input_var_names() 
mf6.get_subcomponent_count()
# var_add = 

# try:
#     mf6.finalize()
#     success=True
# except:
#     raise RuntimeError

In [ ]:
sim = ApiSimulation.load(mf6)
len(ApiSimulation.load(mf6).solutions)

In [ ]:
model = sim.get_model(sim.models[1].name)
model.solution_id


In [ ]:
# current_time, end_time

# all_vars
df_vars = pd.DataFrame(all_vars)[0].str.split('/', expand=True)
df_vars['addr'] = all_vars
# df_vars.to_csv('vars.csv')

In [ ]:
welaux = mf6.get_value(mf6.get_var_address("AUXVAR", 'GWF/WEL'))
welaux[0][-1] = 0
mf6.set_value('GWF/WEL/AUXVAR', welaux)
# mf6.get_value(mf6.get_var_address("x", 'SODIUM')) = val

In [ ]:
mf6.get_value(mf6.get_var_address("AUXVAR", 'GWF/WEL'))
# mf6.finalize()

In [ ]:
welaux = mf6.get_value(mf6.get_var_address("AUXVAR", 'GWF/WEL'))
welaux[0][-1] = 100
# mf6.set_value("AUXVAR", 'GWF/WEL')
mf6.set_value('GWF/WEL/AUXVAR', welaux)

In [ ]:

mf6 = modflowapi.ModflowApi(dll, working_directory = sim_ws)
mf6.initialize()

sim_start = datetime.now()
print("...starting transport solution at {0}".format(sim_start.strftime(DT_FMT)))
# reset the node tracking containers

# get the current sim time
ctime = mf6.get_current_time()
ctimes = [0.0]
# get the ending sim time
etime = mf6.get_end_time()
# max number of solution iterations
max_iter = mf6.get_value(mf6.get_var_address("MXITER", "SLN_1"))
num_fails = 0
# let's do it!
while ctime < etime:
    sol_start = datetime.now()
    # length of the current solve time
    dt = mf6.get_time_step()
    # prep the current time step
    mf6.prepare_time_step(dt)
    kiter = 0
    # prep to solve
    mf6.prepare_solve(1)
    # the one-based stress period number
    stress_period = mf6.get_value(mf6.get_var_address("KPER", "TDIS"))[0]
    time_step = mf6.get_value(mf6.get_var_address("KSTP", "TDIS"))[0]

    # solve until converged
    while kiter < max_iter:
        # apply whatever change we want here
        # val = mf6.get_value(mf6.get_var_address("x", 'SODIUM'))
        # val += 1
        convg = mf6.solve(1)
        if convg:
            td = (datetime.now() - sol_start).total_seconds() / 60.0
            print("transport stress period,time step {0},{1} converged with {2} iters, took {3:10.5G} mins".format(stress_period, time_step, kiter,td))
            break
        kiter += 1

    if not convg:
        td = (datetime.now() - sol_start).total_seconds() / 60.0
        print("transport stress period,time step {0},{1} did not converged, {2} iters, took {3:10.5G} mins".format(
            stress_period, time_step, kiter, td))
        num_fails += 1
mf6.finalize()


In [ ]:
def callback_function(sim, callback_step):
    """
    A demonstration function that dynamically adjusts recharge
    and pumping in a modflow-6 model through the MODFLOW-API

    Parameters
    ----------
    sim : modflowapi.ApiSimulation
        A simulation object for the solution group that is
        currently being solved
    step : enumeration
        modflowapi.Callbacks enumeration object that indicates
        the part of the solution modflow is currently in.
    """
    ml = sim.get_model()
    if callback_step == Callbacks.initialize:
        print(sim.models)

    # if callback_step == Callbacks.stress_period_start:
    #     # adjust recharge for stress periods 7 through 12
    #     if sim.kper <= 6:
    #         rcha = ml.rcha_0
    #         spd = rcha.stress_period_data
    #         print(f"updating recharge: stress_period={ml.kper}")
    #         spd["recharge"] += 0.40 * sim.kper

    if callback_step == Callbacks.timestep_start:
        print(
            f"Testing: stress_period={ml.kper}, timestep={ml.kstp}", ml.kstp
        )
        if ml.kstp != 2:
            ml.wel.stress_period_data["q"] *= 1/ml.kstp
            # ml.wel.stress_period_data['CONCENTRATION'] = 100000000
            # print('..........................', ml.wel.stress_period_data['CONCENTRATION'])
        else:
            ml.wel.stress_period_data['q'] = 0
        # if ml.kstp ==2:
            
    if callback_step == Callbacks.iteration_start:
        # we can implement complex solutions to boundary conditions here!
        pass
    # return wel

# modflowapi.run_simulation(dll, sim_ws, callback_function, verbose=True)


In [ ]:
ucn = flopy.utils.HeadFile(os.path.join("model",'engesgaard1992',"sodium.ucn"),text="concentration")
carr = ucn.get_alldata()

In [ ]:
# carr[0][0]
carr[-1,0,0,:]

# PHREEQCRM TEST

In [ ]:
import phreeqcrm
import numpy as np
import sys
import pandas as pd
from pathlib import Path
import os
import modflowapi
from modflowapi import Callbacks
from workflow import *
from datetime import datetime


In [ ]:
print_selected_output_on = True
print_selected_output_on

In [ ]:
nxyz = 50
nthreads = 1
phreeqc_rm = phreeqcrm.PhreeqcRM(nxyz, nthreads)

status = phreeqc_rm.SetComponentH2O(False)
phreeqc_rm.UseSolutionDensityVolume(False)
status = phreeqc_rm.SetFilePrefix("engesgaard1992")
phreeqc_rm.OpenFiles()
poro = np.full((nxyz), .1)
status = phreeqc_rm.SetPorosity(poro)

print_chemistry_mask = np.full((nxyz), 1)
status = phreeqc_rm.SetPrintChemistryMask(print_chemistry_mask)
nchem = phreeqc_rm.GetChemistryCellCount()
nchem

# init_file = 'phinp.dat'
init_file = 'advect.pqi'

# Load database
databasews = os.path.join("database", "pht3d_datab.dat")
status = phreeqc_rm.LoadDatabase(databasews)
status = phreeqc_rm.RunFile(True, True, True, init_file)# Clear contents of workers and utility
input = "DELETE; -all"
status = phreeqc_rm.RunString(True, False, True, input)

ncomps = phreeqc_rm.FindComponents()
ncomps

components = phreeqc_rm.GetComponents()
components

for comp in components:
    phreeqc_rm.OutputMessage(comp)
phreeqc_rm.OutputMessage("\n")

In [ ]:


for comp in components:
    phreeqc_rm.OutputMessage(comp)
phreeqc_rm.OutputMessage("\n")

ic1 = [-1] * nxyz * 7 
ic1
for i in range(nxyz):
    ic1[i]            =  1  # Solution 1
    ic1[nxyz + i]     = 1  # Equilibrium phases none
    ic1[2 * nxyz + i] =  -1  # Exchange 1
    ic1[3 * nxyz + i] = -1  # Surface none
    ic1[4 * nxyz + i] = -1  # Gas phase none
    ic1[5 * nxyz + i] = -1  # Solid solutions none
    ic1[6 * nxyz + i] = -1  # Kinetics none

status = phreeqc_rm.InitialPhreeqc2Module(ic1)

# Initial equilibration of cells
time = 0.0
time_step = 0.0
status = phreeqc_rm.SetTime(time)
status = phreeqc_rm.SetTimeStep(time_step)
status = phreeqc_rm.RunCells()


In [ ]:
status

c_dbl_vect = phreeqc_rm.GetConcentrations()


In [ ]:
#ncomp * ncell

ncomps*nxyz, components


In [ ]:
phreeqc_rm.GetSpeciesNames()
phreeqc_rm.GetSpeciesConcentrations()

In [ ]:
phreeqc_rm.GetSpeciesNames()


In [ ]:
phreeqc_rm.GetDatabaseFileName()
components
phreeqc_rm.GetEquilibriumPhases()
# phreeqc_rm.SetConcentrations()

In [ ]:
components

In [ ]:
# status
c_dbl_vect = phreeqc_rm.GetConcentrations()

In [ ]:
nlay = 1  # Number of layers
Lx = 0.5 #m
ncol = 50  # Number of columns
nrow = 1  

n=nxyz

conc = [c_dbl_vect[i:i + n] for i in range(0, len(c_dbl_vect), n)]

sconc = {}
for e, c in enumerate(components):
    sconc[c] = np.reshape(conc[e], (nlay, nrow, ncol))
sconc

for c in components:
    print(sconc[c].dtype)
# sconc


In [ ]:
ic1 = [-1] * 50 * 7 
len(ic1)

In [ ]:
# sim = build_model(ws = 'model', sim_name = 'engesgaard1992', spls = components, sconc=sconc)

# strt

In [ ]:
components

In [ ]:
def flatten_list(xss):
    return [x for xs in xss for x in xs]


In [ ]:
# sim_ws = os.path.join('model', '1dtest', 'chloride')
sim_ws = Path("model/engesgaard1992/")
dll = Path("bin/win/libmf6")

mf6 = modflowapi.ModflowApi(dll, working_directory = sim_ws)
mf6.initialize()


sim_start = datetime.now()
print("...starting transport solution at {0}".format(sim_start.strftime(DT_FMT)))
# reset the node tracking containers

# get the current sim time
ctime = mf6.get_current_time()
ctimes = [0.0]
# get the ending sim time
etime = mf6.get_end_time()
# max number of solution iterations
max_iter = mf6.get_value(mf6.get_var_address("MXITER", "SLN_2"))
num_fails = 0
# let's do it!
while ctime < etime:
    sol_start = datetime.now()
    # length of the current solve time
    dt = mf6.get_time_step()
    # prep the current time step
    mf6.prepare_time_step(dt)
    kiter = 0
    # prep to solve
    mf6.prepare_solve(1)
    # the one-based stress period number
    stress_period = mf6.get_value(mf6.get_var_address("KPER", "TDIS"))[0]
    time_step = mf6.get_value(mf6.get_var_address("KSTP", "TDIS"))[0]

    # array to store transported components
    print(f'\nGetting concentration arrays --- time step: {time_step} --- elapsed time: {ctime}')
    mf6_conc_array = [mf6.get_value(mf6.get_var_address("X", f'{c.upper()}')) for c in components]
    c_dbl_vect = flatten_list(mf6_conc_array)

    phreeqc_rm.SetScreenOn(True)
    print_selected_output_on = (ctime == etime)
    print_chemistry_on = (ctime == etime)
    status = phreeqc_rm.SetSelectedOutputOn(True)
    status = phreeqc_rm.SetPrintChemistryOn(print_chemistry_on, False, False) 

    status = phreeqc_rm.SetConcentrations(c_dbl_vect)  
    message = 'Beginning reaction calculation               {} days\n'.format(ctime)
    phreeqc_rm.LogMessage(message)
    phreeqc_rm.ScreenMessage(message)
    status = phreeqc_rm.RunCells()

    # Transfer data from PhreeqcRM for transport
    #status = phreeqc_rm.GetConcentrations(c_dbl_vect)
    c_dbl_vect = phreeqc_rm.GetConcentrations()

    conc = [c_dbl_vect[i:i + n] for i in range(0, len(c_dbl_vect), n)]
    sconc = {}
    for e, c in enumerate(components):
        sconc[c] = np.reshape(conc[e], (nlay, nrow, ncol))

    # solve transport until converged
    for c in components:
        print(f'\nSolving for component: {c}')
        while kiter < max_iter:
            # print(list(mf6_conc_array))
            if c == 'mg':
                mf6.set_value(f'{c.upper()}/X', sconc[c]+100)
            else:
                mf6.set_value(f'{c.upper()}/X', sconc[c])
            convg = mf6.solve(1)
            if convg:
                td = (datetime.now() - sol_start).total_seconds() / 60.0
                print("Transport stress period: {0} --- time step: {1} --- converged with {2} iters --- took {3:10.5G} mins".format(stress_period, time_step, kiter,td))
                break
            kiter += 1

    if not convg:
        td = (datetime.now() - sol_start).total_seconds() / 60.0
        print("transport stress period {0}, time step {1}, did not converged, {2} iters, took {3:10.5G} mins".format(
            stress_period, time_step, kiter, td))
        num_fails += 1
    try:
        mf6.finalize_solve(1)
    except:
        pass
    mf6.finalize_time_step()
    # update the current time tracking
    ctime = mf6.get_current_time()
# sim = flopy.mf6.MFSimulation.load('engesgaard1992', 'mf6', 'mf6' , os.path.join('model', 'engesgaard1992'))
# plot_concentrations(sim)
sim_end = datetime.now()
td = (sim_end - sim_start).total_seconds() / 60.0
print("\n...transport solution finished at {0}, took: {1:10.5G} mins".format(sim_end.strftime(DT_FMT),td))
if num_fails > 0:
    print("...failed to converge {0} times".format(num_fails))
print("\n")
status = phreeqc_rm.CloseFiles()
status = phreeqc_rm.MpiWorkerBreak()
mf6.finalize()

In [ ]:
[c_dbl_vect[i:i + n] for i in range(0, len(c_dbl_vect), n)]
sconc

In [ ]:
sim = flopy.mf6.MFSimulation.load('engesgaard1992', 'mf6', 'mf6', sim_ws, verbosity_level=0)

sim.model_names
plot_concentrations(sim)

# Engesgaard 1992 - PHT3D

In [ ]:
from pathlib import Path
import os
from modflowapi.extensions import ApiSimulation
from modflowapi import Callbacks
from workflow import *
from datetime import datetime

DT_FMT = "%Y-%m-%d %H:%M:%S"


In [ ]:
wd = os.path.join('benchmark', 'pht3d')
dx = 0.01
simdf = pd.read_csv(os.path.join(wd, 'out.dat'), sep = '\t', skipinitialspace=True, index_col=[0])
simdf.drop(simdf.columns[len(simdf.columns)-1], axis=1, inplace=True)
# simdf.columns = [col.lower() for col in simdf.columns]
simdf.loc[:, 'x'] = simdf['cell'] * dx 
display(simdf)

Lx = 0.5 #m
ncol = 50  # Number of columns
delr = Lx/ncol

simapi = pd.read_csv(os.path.join( 'sout.csv'), sep = ',', skipinitialspace=True, index_col=[0])
simapi.loc[:, 'x'] = (simapi['cell'] + 1)*delr

simapi['time_d'] = list(simdf.index)*1
simapi.index = simapi['time_d'] 
simapi

In [ ]:
# list(simdf.index)*10

In [ ]:
fig, axs = plt.subplots(4,2, figsize = (10, 10))
t = 0.24
for var, ax in zip(simdf.iloc[:,1:-1 ].columns, axs.flatten()):
    # ax.plot(simdf.x.unique(), simdf.loc[t, var], label = 'Engesgaard1992 - PHT3D')
    ax.plot(simapi.x.unique(), simapi.loc[t, var], label = 'Engesgaard1992 - MF6RTM')
    ax.set_xlabel('distance (m)')
    if var not in ['ph', 'pe']:
        ax.set_ylabel('C (mol L$^{-1}$)')
    ax.set_title(f'{var}')
    ax.legend()
fig.tight_layout()

In [ ]:
simdf.loc[0.24, ['x','Calcite']].pivot_table(index='x', columns = 'time_d').plot();
simdf.loc[0.24, ['x','Mg']].pivot_table(index='x', columns = 'time_d').plot() 

In [ ]:
# C_4 = loadtxt('PHT3D001.ACN')
# Ca = loadtxt('PHT3D002.ACN')
# Cl = loadtxt('PHT3D003.ACN')
# Mg = loadtxt('PHT3D004.ACN')
# pH = loadtxt('PHT3D005.ACN')
# pe = loadtxt('PHT3D006.ACN')



In [ ]:
ucn_dic = {'c':1,'ca':2, 'cl':3, 'mg':4, 'h':5, 'charge':6}

comps = list(ucn_dic.keys())
# comps.extend(['h2o'])

# comps = [ 'h', 'c']

for i in [0, 10, 15, 23]:
    sim_ws = os.path.join('model', 'engesgaard1992api')
    wd = os.path.join('benchmark', 'pht3d')

    sim = flopy.mf6.MFSimulation.load('engesgaard1992api', 'mf6', 'mf6', sim_ws, verbosity_level=0)
    nrow = len(comps)
    fig, axs = plt.subplots(nrow, 1, figsize=(3.3,8.2))

    for c, ax in zip(comps, axs.flatten()):
        gwf = sim.get_model(c)

        ucn = flopy.utils.HeadFile(os.path.join("model",sim.name,f"{c}.ucn"),text="concentration")
        results = ucn.get_alldata()
        times = gwf.output.concentration().get_times()

        ucn_pht = flopy.utils.HeadFile(os.path.join(wd,f"PHT3D00{ucn_dic[c]}.ucn"),text="concentration")
        results_pht = ucn_pht.get_alldata()
        times_pht = ucn_pht.get_times()
        
        # ax.plot(np.log(concentration_m3_to_l(results[i, 0, 0,:])), label = 'mf6api')
        ax.set_title(f"{c}")
        # ax.plot(times, concentration_m3_to_l(results[:, 0, 0,-1]), label = 'mf6api')
        # ax.plot(times_pht, results_pht[:, 0, 0,25], label = 'pht3d')
        ax.plot(results_pht[i, 0, 0,:], label = 'pht3d')
        if c in ['h3', 'charge']:
            # ax.plot(np.log(results_pht[i, 0, 0,:]), label = 'pht3d')
            ax.plot(np.log(concentration_m3_to_l(results[i, 0, 0,:])), label = 'mf6api')
            # print(np.log(results_pht[i, 0, 0,:]))
        else:
            ax.plot(concentration_m3_to_l(results[i, 0, 0,:]), label = 'mf6api')
        ax.set_ylabel('Conc (mol/L)')
        ax.set_xlabel('Time')
        ax.legend()
    fig.tight_layout()
    plt.show()

In [ ]:
comps

# Engesgaard 1992 - MF6RTM

In [ ]:
sim_ws = os.path.join('model', 'engesgaard1992')
sim = flopy.mf6.MFSimulation.load('engesgaard1992', 'mf6', 'mf6', sim_ws, verbosity_level=0)

comp = sim.model_names[1:]
comp = [c.capitalize() for c in comp if c !='o']
comp

dist_arr = np.arange(0, 50*dx, dx)
gwf = sim.get_model('gwf')

fig, axs = plt.subplots(2, 3, figsize=(6.3,3.2))
for ax, c in zip(axs.flatten(), comp):
        ucn = flopy.utils.HeadFile(os.path.join("model",sim.name,f"{c}.ucn"),text=f"concentration")
        results = ucn.get_alldata()
        # results = gwf.output.concentration().get_alldata()
        times = gwf.output.concentration().get_times()

        tf = times[-1]
        r_tf = results[0, 0, 0,:]
        ax.plot(dist_arr, r_tf)

fig.tight_layout()

In [ ]:
comp

In [ ]:
dist_arr = np.arange(0, 50*dx, dx)
plt.plot(dist_arr, results[-1, 0, 0,:])
len(dist_arr)